# NeRF-WandB-COLAB

### [NeRF 공식 저장소](https://github.com/bmild/nerf)를 수정하여 [wandb 연동이 편리하도록 만든 저장소](https://github.com/ProtossDragoon/nerf-wandb.git)의 소스코드를 이용해 커스텀 데이터셋에 NeRF 모델 학습

Plank-ing Hyundong 3D Reconstruction Project
Created 2022.07.12 <br>

**NoteBook Author** <br>
[Janghoo Lee](https://www.linkedin.com/in/janghoo-lee-25212a1a0) <br>
🎓 : [ProtossDragoon](https://github.com/ProtossDragoon) <br>
📧 : dlwkdgn3@gmail.com <br>

🤔 If you have any questions, please raise an issue in our [github repo](https://github.com/ProtossDragoon/PlankHyundong).

## 환경

### TF 런타임 변경

In [ ]:
%tensorflow_version 1.x

### 의존 패키지 설치

In [ ]:
!sudo apt -qq install imagemagick
!pip install ConfigArgParse -qqq
!pip install imageio-ffmpeg -qqq

### 저장소

In [ ]:
!git clone https://github.com/ProtossDragoon/nerf-wandb.git

In [ ]:
%cd nerf-wandb
#!git checkout dev # 개발 중
!ls -al

**wandb**

- 실험 관리 도구로 wandb 를 사용합니다.
- [plank-hyundong](https://wandb.ai/plank-hyundong) 팀의 [wandb 프로젝트 plank-hyundong](https://wandb.ai/plank-hyundong/plank-hyundong) 관리자에게 접근 키를 문의하세요.

In [ ]:
!pip install wandb -qqq

In [ ]:
# Log in to your W&B account    
import wandb
wandb.login()

## 커스텀 데이터로 실행

- LLFF 를 통해 얻은 pose 가 필요합니다.

### 환경

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%y%m%d_%H%M%S')

dataset_name = 'hyundong360_50' #@param ['hyundong360_50', 'hyundong360', 'hyundong360_100', 'hyundong360_400', 'noparking360', 'noparking360_100', 'noparking360_200', 'noparking360_400']
downsample_factor = 8 #@param {type:"slider", min:1, max:64, step:1}
netdepth = 8 #@param {type:"slider", min:4, max:16, step:2}
netwidth = 128 #@param {type:"slider", min:64, max:256, step:4}
n_rays = 8192 #@param {type:"slider", min:1024, max:131072, step:1024}
n_pts = 65536 #@param {type:"slider", min:2048, max:262144, step:1024}
experiment_name = f'{dataset_name}_{downsample_factor}_downsampled_{now}'
max_iter = 50000 #@param
learning_rate = 0.0001 #@param
video_saving_cnt = 10 #@param {type:"slider", min:1, max:10, step:1}
n_samples = 64 #@param {type:"slider", min:32, max:256, step:32}
# fine 모델에서 사용되는 샘플 개수는 coarse 모델의 sampling 개수의 2배로 설정한다.
# 공식 논문에서 제안하는 대로, 64이면 128.
n_importance = n_samples * 2

In [ ]:
_dummy_dir = f'./logs/{experiment_name}'
_tensorboard_logdir = f'./logs/summaries/{experiment_name}'
print(f'experiment: {experiment_name}')

### 실행

**플래그 해석**

- no_ndc flag: do not use normalized device coordinates
- spherify: set for spherical 360 scenes
- lindisp: sampling linearly in disparity rather than depth

In [ ]:
!python run_nerf.py \
    --wandbproject {'plank-hyundong'} \
    --wandbentity {'plank-hyundong'} \
    --maxiter {max_iter} \
    --datadir /content/drive/MyDrive/dev/llff_data/{dataset_name} \
    --dataset_type llff \
    --factor {downsample_factor} \
    --netdepth {netdepth}\
    --netwidth {netwidth} \
    --netdepth_fine {netdepth} \
    --netwidth_fine {netwidth} \
    --chunk {n_rays} \
    --netchunk {n_pts} \
    --lrate {learning_rate} \
    --i_video {max_iter // video_saving_cnt - 1} \
    --expname {experiment_name} \
    --N_samples {n_samples} \
    --N_importance {n_importance} \
    --no_ndc \
    --spherify \
    --lindisp \